# The Delta Method Applied to the Sharpe Ratio

In [1]:
include("jlFiles/printmat.jl")
include("jlFiles/NWFn.jl")

NWFn

In [2]:
xx   = readdlm("Data/FFmFactorsPs.csv",',',header=true)
x    = xx[1]
Rme  = x[:,2]/100
T    = size(Rme,1)

388

## Point Estimates of Mean and 2nd Moment

Estimate the mean and second moment as usual 

$$
\begin{align*}
\hat{\mu}  & =\frac{1}{T}\sum_{t=1}^{T}x_{t}\\
\hat{\mu}_{2}  & =\frac{1}{T}\sum_{t=1}^{T}x_{t}^{2}
\end{align*}
$$

Use GMM to estimate the covariance matrix ($V$) of $(\hat{\mu},\hat{\mu}_{2})$

In [3]:
mu    = mean(Rme,1)                  #GMM to estimate mean and 2nd moment
mu_2  = mean(Rme.^2,1)
println("\n","[mu mu_2]")
printmat([mu mu_2])

g    = [(Rme .- mu) (Rme.^2 .- mu_2)]    #moment conditions
T    = size(g,1)
gbar = vec(mean(g,1))                    #vector
println("Sample moment conditions, gbar ")
printmat(gbar)

S = NWFn(g,1)                     #Var[sqrt(T)*gbar], Newey-West
D = -1
V = inv(D*inv(S)*D')              #Var[sqrt(T)*(mu,mu_2)]
println("Cov(params)*10000")
printmat(V*10000)


[mu mu_2]
     0.006     0.002

Sample moment conditions, gbar 
     0.000
    -0.000

Cov(params)*10000
    23.167    -0.770
    -0.770     0.198



## The Sharpe Ratio and Its Derivatives

The Sharpe ratio and its derivatives (with respect to the parameters of the
Sharpe ratio) are

$$
\begin{align*}
\frac{\operatorname*{E}(x)}{\sigma(x)} &  =\frac{\mu}{(\mu_{2}
-\mu^{2})^{1/2}},\ \text{where}\ \beta=(\mu,\mu_{2})
\end{align*}
$$

Let $f(\beta)$ denote the Sharpe ratio where $\beta$ is a vector of parameters 
consisting of the mean and the second moment ($\mu,\mu_{2}$). The derivates are then

$$
\begin{align*}
\frac{\partial f(\beta)}{\partial\beta^{\prime}} &  =\left[
\begin{array}
[c]{cc}
\frac{\mu_{2}}{(\mu_{2}-\mu^{2})^{3/2}} & \frac{-\mu}{2(\mu_{2}-\mu^{2})^{3/2}}
\end{array}
\right]  
\end{align*}
$$

In [4]:
function SRFn(par)
# SRFn    Calculates the Sharpe ratio from the mean and 2nd moment
  mu   = par[1]          #E(Z)
  mu_2 = par[2]          #E(Z^2)
  s2   = mu_2 - mu.^2
  SR   = mu./sqrt(s2)
  return SR
end

SRFn (generic function with 1 method)

In [5]:
                                  #delta method
df = [ (mu_2./(mu_2 - mu.^2).^(3/2)) (-mu./(2*(mu_2 - mu.^2).^(3/2)))]
println("\nAnalytical derivatives ")
printmat(df)


Analytical derivatives 
    22.121   -30.957



## Numerical Derivatives


Sometimes, it is cumbersome to code up the analytical derivatives. We can then
use a simple forward approximation to calculate numerical derivatives.

Consider
vector valued function that takes the vector $\beta=(\beta_{1},\beta
_{2},...,\beta_{k})$ as input and produces the vector $[f_{1}(\beta
),f_{2}(\beta),...,f_{q}(\beta)]$ as output. A very simple code can be
structured as follows: let column $j$ of $\partial f\left(  \beta\right)
/\partial\beta^{\prime}$ be

$$
\left[
\begin{array}[c]{c}
\frac{\partial f_{1}\left(  \beta\right)  }{\partial\beta_{j}}\\
\vdots\\
\frac{\partial f_{q}\left(  \beta\right)  }{\partial\beta_{j}}
\end{array}
\right]  =\frac{f(\tilde{\beta})-f(\beta)}{\Delta}\ \text{, where } \ \tilde{\beta
}=\beta \ \text{ except that } \ \tilde{\beta}_{j}=\beta_{j}+\Delta.
$$

However, you find better methods at http://www.juliadiff.org/

In [6]:
function NumJac(fun,b0,h=1e-7)           #crude function for numerical derivatives 
    k  = length(b0)  
    y0 = fun(b0)                     
    Jac   = fill(NaN,(length(y0),k))
    for j = 1:k              
        b1    = b0 + 0.0 
        b1[j] = b1[j] + h
        Jac[:,j]  = (fun(b1)- y0)/h
    end    
    return Jac
end

NumJac (generic function with 2 methods)

In [7]:
par0 = [mu;mu_2]

Jac = NumJac(SRFn,par0)

println("\nNumerical derivatives, just for comparison ")
printmat(Jac)


Numerical derivatives, just for comparison 
    22.121   -30.956



## Applying the Delta Method to the Sharpe Ratio


Recall that if

$$
\sqrt{T}(\hat{\beta}-\beta_{0})\overset{d}{\rightarrow}N\left(  0,V_{k\times k}\right)  ,
$$
then the distribution of the function $f(\hat{\beta})$ is

$$
\sqrt{T}[f(\hat{\beta})-f\left(  \beta_{0}\right)  ]\overset{d}{\rightarrow
}N\left(  0,\Lambda_{q\times q}\right)  
$$

with 

$$
\begin{align*}
\Lambda &  =\frac{\partial f\left(  \beta_{0}\right)  }{\partial
\beta^{^{\prime}}}V\frac{\partial f\left(  \beta_{0}\right)  ^{\prime}
}{\partial\beta}, \ \text{where } \ \frac{\partial f\left(  \beta\right)
}{\partial\beta^{\prime}}=\left[
\begin{array}[c]{ccc}
\frac{\partial f_{1}\left(  \beta\right)  }{\partial\beta_{1}} & \cdots &
\frac{\partial f_{1}\left(  \beta\right)  }{\partial\beta_{k}}\\
\vdots & \ddots & \vdots\\
\frac{\partial f_{q}\left(  \beta\right)  }{\partial\beta_{1}} & \cdots &
\frac{\partial f_{q}\left(  \beta\right)  }{\partial\beta_{k}}
\end{array}
\right]  _{q\times k}
\end{align*}
$$


In [8]:
SR0 = SRFn(par0)

Std_SR = sqrt.(df*V*df'/T)
println("\nSR and its Std ")
printmat([SR0 Std_SR])

println("SR and 90% conf band: ")
printmat([SR0 (SR0-1.65*Std_SR) (SR0+1.65*Std_SR)])


SR and its Std 
     0.131     0.057

SR and 90% conf band: 
     0.131     0.037     0.225

